In [32]:
import os
import json
import pandas as pd
import numpy as np
import requests
from modules.transcript_dataset import TranscriptDataset

In [33]:
# Please set a valid Google API
# Please download the 'data' folder from https://drive.google.com/drive/folders/1D-OHX0LUNA5Y-0h8UXwqa4H_yETZN_t_ to your project root directory

GOOGLE_API_KEY = '********************'

In [34]:
dataset_builder = TranscriptDataset(api_key=GOOGLE_API_KEY)

# dataset_folder => The folder to load / save the datasets file from. If not exists it will create it automatically
# videos_count => Total number of videos to include in the dataset. If there are already saved videos in the "data" folder,
# it will fill the remaining videos count (For example if the parameter: videos_count is 4000 and there are already 1500 videos
# in the folder, it will fetch 2500 videos). Default is 100
# country_code => 2 digits ISO Code of the country you want to fetch videos from (If not set, the default is United States 'us')
# Relevant codes for countries with a majority of English based videos => 'ca' Canada, 'uk' United Kingdom, 'au' Australia, 'in' India
#  'nz' New Zealand, 'za' South Africa
# language => 2 digits ISO Code of the video's language you want to use (If not set, the default is English 'en')
dataset_builder.build_dataset(dataset_folder='data', videos_count=3500, country_code='uk')

# dataset_folder => The folder to load / save the datasets file from. If not exists it will create it automatically
# comments_count => The amount of comments to fetch for each video. Default is 100
dataset_builder.build_comments(dataset_folder='data', comments_count=200)

There are currnet 3500 videos in 7 files
There are already at least 3500. No need to do anything
Working on file dataset_1.json
Fetching comments of MMbdcJhNxtI
Fetching comments of TgWyEsZX48s
Fetching comments of US4nNVZ7wO8
Fetching comments of _V6TZulh1G0
Fetching comments of f5tbu9KX7j4
Fetching comments of kTrhlDVdIoQ
Fetching comments of tLSeYR6qOlM
Fetching comments of wC4VE-C8ziw
Working on file dataset_0.json
Fetching comments of Cdp4hcU9n18
Fetching comments of PEBvkz9uW-s
Fetching comments of UFAAppCA7wM
Video UFAAppCA7wM not found. Returning empty list
Fetching comments of afE1vCqgKlM
Fetching comments of c09mLX2YDyc
Fetching comments of ozLZgdAGaiI
Fetching comments of pR0ZFiCtCM4
Working on file dataset_6.json
Fetching comments of -6YbcUu0Jf4
Fetching comments of 0YYA76ckoHo
Fetching comments of 0fn8TcpY1N0
Fetching comments of 12F1cgmfpvQ
Fetching comments of 1za3s-qdqCc
Fetching comments of 3QPjSJDVe3M
Fetching comments of 4d6RtKu5SxY
Fetching comments of 4htS78fnqWY
F